In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import scipy.spatial.distance as dis
from scipy.ndimage import filters
from sklearn.feature_extraction.text import CountVectorizer
import gc

In [3]:
def getDistance(abp_df,level):
    try:
        col_cfg = pd.DataFrame(
                    np.array(
                        [['BedType','Dice',1],
                         ['RoomSize','cityblock',0.5],
                         ['RoomView','Dice',1],
                         ['Breakfast','Dice',1]
                       ]),
                    columns = ['name','algo','weight'])
        col_cfg = col_cfg.set_index('name')

        rows = abp_df.drop(['HotelID','HotelName','RoomTypeID','RoomName','Price','RatePlanID'], axis=1)

        rows['RoomSize'] = rows['RoomSize'].apply(lambda x:re.search("([0-9])+",str(x)).group(0))

        d_list = []

        for c in rows.columns:
            algo = col_cfg.loc[c]['algo']
            if algo == 'Dice':
                one_hot = MultiLabelBinarizer()
                d_list.append(pd.DataFrame(dis.pdist(one_hot.fit_transform(rows[c].apply(lambda x:tuple(str(x).split(',')))),algo)))
            elif algo == 'cityblock':
                ud = dis.pdist(rows[c].values.reshape(-1,1),algo).reshape(-1,1)
                scaler = MinMaxScaler()
                scaler.fit(ud)
                d_list.append(pd.DataFrame(scaler.transform(ud)))
            elif algo == 'ngram':
                corpus = rows[c]
                v = CountVectorizer(ngram_range=(1, 3), binary = True, lowercase = True)
                d_list.append(pd.DataFrame(dis.pdist(v.fit_transform(corpus).toarray(),'Dice')))
            else:
                print('error')

        dm = pd.concat(d_list, ignore_index=True, axis=1)
        dm.columns = rows.columns

        d_weight = col_cfg['weight'].values.astype(np.float)
        test = dm.values * d_weight
        ag1 = (dm.values * d_weight).mean(axis=1)
        ag1_sq = dis.squareform(ag1)
        gaussian_filter(ag1_sq)
        np.fill_diagonal(ag1_sq,1)



    #     ag1_sq[ag1_sq==0] = 1
        distance_df = pd.DataFrame(ag1_sq)
        print(distance_df)
        result = []
        for row_index, row in distance_df.iterrows():
            for col_index,distance in row.iteritems():
                if distance < level:
                    result.append([str(abp_df.loc[row_index].RatePlanID),str(abp_df.loc[col_index].RatePlanID),distance])
        return pd.DataFrame(np.array(result),columns=['root','child','distance'])
    except ValueError as e:
        print(e)
        raise Exception("Calculate failed!")